# Kymberly Ayodeji: Solution code below

In [1]:
 # Install Necessary resources 
!pip install nltk
!pip install TextBlob
!pip install Beautifulsoup4
!pip install googletrans==4.0.0-rc1
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Import Needed Libraries and Necessary resources from NLTK, Beautiful Soup, TextBlob, Google Translate
import html5lib
from datetime import datetime
from bs4 import BeautifulSoup
from textblob import TextBlob
import numpy as np
import pandas as pd
import requests
import io
import nltk
from nltk import word_tokenize
from googletrans import Translator 
import os
import sqlite3 as sql


# 1) Extraction

In [3]:
#1a) Fetch Financial News headlines
cnbc_URL = "https://www.cnbc.com/stocks/"
cnbc_headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
cnbc_r = requests.get(cnbc_URL, headers=cnbc_headers)
cnbc_timestamp=datetime.now()



In [4]:
#1b)Fetch Stock Market Data 
aapl_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_aapl=requests.get(aapl_url)
msft_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_msft=requests.get(msft_url)
goog_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOOG&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_goog=requests.get(goog_url)
amzn_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AMZN&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_amzn=requests.get(amzn_url)
meta_url='https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=META&interval=60min&outputsize=compact&apikey=LJ7H7F9D9Z7ECUK7&datatype=csv'
r_meta=requests.get(meta_url)

# 2) Transformation

## 2a, 2b, 2c, 2d: Financial News Headline Data 

In [5]:
# Using the extracted Financial News headline data, retrieve the url for the headline
html_text = cnbc_r.text
soup = BeautifulSoup(html_text, 'lxml')
card_titles=soup.find_all('a', class_='Card-title')
headlines_data=[]
for title in card_titles:
    headlines_data.append(title['href'])


In [6]:
# Extract title from url and strip .html then create a list of sentences for each url 
df_holder=pd.DataFrame(headlines_data, columns=['url'])
df_holder[['http','empty', 'site', 'year','month','date','title']] = df_holder.url.str.split("/",expand=True)
titles=df_holder['title'].str.replace('-', ' ').tolist()
sentences =  [x[:-4] for x in titles]



In [7]:
#Create a data frame for the headlines and add the columns for title, url, timestamp
df_headlines=pd.DataFrame(sentences, columns=['headline'])
df_headlines['url']=df_holder['url']
df_headlines['timestamp'] = cnbc_timestamp

In [8]:
#2a. Add the sentiment score and sentiment for each headline sentence using the toekenize functions
df_headlines['sentiment_score']=df_headlines['headline'].apply(lambda sent: TextBlob(sent).sentiment.polarity)
def f(score):
    if -1 <= score  and score < -0.2: return "negative"
    elif -0.2 <= score and score > 0.2: return "neutral" 
    else: return "positive"
assign_sentiment = lambda x: list(map(f, x))
df_headlines['sentiment']=assign_sentiment(df_headlines['sentiment_score'])
      
    

In [9]:
#2b. Tokenize the text into individual "relevant words" by NLTK
df_headlines['relevant_words']=df_headlines['headline'].apply(word_tokenize) 

In [10]:
#2c. Create new translated column
my_translator = Translator()
df_headlines['headline_es']=df_headlines['headline'].apply(lambda x: my_translator.translate(x, src='en', dest='es').text)
df_headlines['headline_it']=df_headlines['headline'].apply(lambda x: my_translator.translate(x, src='en', dest='it').text)


In [11]:
#View Data
df_headlines.index.name = "ID"
df_headlines.head(5)

,headline,url,timestamp,sentiment_score,sentiment,relevant_words,headline_es,headline_it
ID,,,,,,,,
0,delta is the most overbought stock on wall str...,https://www.cnbc.com/2023/06/18/delta-is-the-m...,2023-06-18 22:12:00.557919,0.500000,neutral,"[delta, is, the, most, overbought, stock, on, ...",Delta es el stock más exagerado en Wall Street...,Delta è lo stock più comprovato di Wall Street...
1,this compact fund beats the market and provide...,https://www.cnbc.com/2023/06/18/this-compact-f...,2023-06-18 22:12:00.557919,-0.388889,negative,"[this, compact, fund, beats, the, market, and,...",Este fondo compacto supera al mercado y propor...,Questo fondo compatto batte il mercato e forni...
2,blinken meets chinese foreign minister qin gan...,https://www.cnbc.com/2023/06/18/blinken-meets-...,2023-06-18 22:12:00.557919,0.011667,positive,"[blinken, meets, chinese, foreign, minister, q...",Blinken se reúne con el ministro de Relaciones...,Blinken incontra il ministro degli Esteri cine...
3,the japanese equity rally could broaden out so...,https://www.cnbc.com/2023/06/17/the-japanese-e...,2023-06-18 22:12:00.557919,-0.083333,positive,"[the, japanese, equity, rally, could, broaden,...",El rally de equidad japonés podría ampliar alg...,Il rally azionario giapponese potrebbe ampliar...
4,the ipo market may put pressure on the fed cra...,https://www.cnbc.com/2023/06/16/the-ipo-market...,2023-06-18 22:12:00.557919,0.000000,positive,"[the, ipo, market, may, put, pressure, on, the...",El mercado de OPI puede ejercer presión sobre ...,Il mercato dell'IPO potrebbe esercitare pressi...


In [12]:
# Create file folders (Made it manually but included the code to make it within the code)
os.makedirs("data")
os.makedirs("data/headlines")

FileExistsError: [Errno 17] File exists: 'data'

In [13]:
# 2d. Save the headlines DataFrame to a CSV file 
df_headlines.index = df_headlines.index.rename('ID')
df_headlines.to_csv('data/headlines/headlines_data.csv', encoding='utf-8')

## 2e, 2f: Stocks Data Transformation

In [14]:
#Read the stock market data 
aapl_content=r_aapl.content
aapl = pd.read_csv(io.StringIO(aapl_content.decode('utf-8')))
msft_content=r_msft.content
msft = pd.read_csv(io.StringIO(msft_content.decode('utf-8')))
goog_content=r_goog.content
goog = pd.read_csv(io.StringIO(goog_content.decode('utf-8')))
amzn_content=r_amzn.content
amzn = pd.read_csv(io.StringIO(amzn_content.decode('utf-8')))
meta_content=r_meta.content
meta = pd.read_csv(io.StringIO(meta_content.decode('utf-8')))

In [15]:
# In prepraration of merge each stock dataframe into one, Lets add the stock name as a column named symbol 
aapl['symbol']='AAPL'
msft['symbol']='MSFT'
goog['symbol']='GOOG'
amzn['symbol']='AMZN'
meta['symbol']='META'

In [16]:
#Merge the stocks into one data frame and rename the columns to match the assignment
df_stocks=aapl
df_stocks.append(msft, ignore_index = True)
df_stocks.append(goog, ignore_index = True)
df_stocks.append(amzn, ignore_index = True)
df_stocks.append(meta, ignore_index = True)
dict={"timestamp":"date", "open":"open_price", "high":"highest_price", "low":"lowest_price", "close":"close_price"}
df_stocks.rename(columns=dict,inplace=True)

In [17]:
df_stocks.index = df_stocks.index.rename('ID')

In [18]:
# 2f. Save Data
#os.makedirs("data/stocks")
df_stocks.to_csv('data/stocks/stocks_data.csv', index=False,encoding='utf-8')
df_stocks.head()

,date,open_price,highest_price,lowest_price,close_price,volume,symbol
ID,,,,,,,
0,2023-06-16 19:00:00,184.92,185.00,184.9200,184.94,9615,AAPL
1,2023-06-16 18:00:00,184.93,184.98,184.9200,184.92,5007,AAPL
2,2023-06-16 17:00:00,185.02,185.04,184.9200,184.93,123445,AAPL
3,2023-06-16 16:00:00,184.92,185.20,184.9015,185.01,15064383,AAPL
4,2023-06-16 15:00:00,184.90,185.35,184.2700,185.01,11935261,AAPL


# 3) Load

In [19]:
# 3a) Create a db with the schema for the headlines table and the stocks table
db = sql.connect('etl_extended_case.db')
c = db.cursor()


### 3b) Create two tables for the headline and stocks data 

In [20]:
c.execute("""CREATE TABLE headlines(ID INTEGER PRIMARY KEY NOT NULL, 
    headline TEXT, 
    url TEXT,
    timestamp TEXT,
    sentiment_score REAL,
    sentiment TEXT, 
    relevant_words TEXT,
    headline_es TEXT, 
    headline_it TEXT)""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 21))



OperationalError: table headlines already exists

In [21]:
c.execute("""CREATE TABLE stocks(ID INTEGER PRIMARY KEY NOT NULL, 
    date TEXT, 
    open_price REAL,
    highest_price REAL,
    lowest_price REAL,
    close_price REAL, 
    volume INTEGER,
    symbol TEXT)""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 16))



OperationalError: table stocks already exists

## Read the data from the CSV files and load into the SQLite database

In [22]:
dfHeadlines = pd.read_csv('data/headlines/headlines_data.csv')
dfHeadlines.to_sql('headlines', db, if_exists='append',index=False)

IntegrityError: UNIQUE constraint failed: headlines.ID

In [23]:
dfStocks = pd.read_csv('data/stocks/stocks_data.csv')
dfStocks.to_sql('stocks', db, if_exists='append',index=False)

In [24]:
#check data

In [25]:
c.execute('''SELECT * FROM headlines''')

In [26]:
c.execute('''SELECT * from stocks''')

In [28]:
#check data
c.execute('''SELECT * FROM headlines LIMIT 5''')

In [29]:
c.execute('''SELECT * FROM stocks LIMIT 5''')

In [30]:
db.commit()
db.close()